# L4: Production-ready techniques

In this lesson, you are going to learn about some production techniques, and integrate them into a web app.

In [2]:
const handler = (req) => {
    // Create a new response object
    const body = new TextEncoder().encode("Hello World!");
    return new Response(body, { status: 200 });
  };

In [3]:
let server = Deno.serve({ port: 8001 }, handler);

Listening on http://0.0.0.0:8001/


In [4]:
const response = await fetch("http://localhost:8001")

In [5]:
await response.text()

"Hello World!"

In [6]:
await server.shutdown()

### Import required dependencies and load the API key

In [ ]:
// import * as mod from "https://deno.land/std@0.213.0/dotenv/mod.ts";
import { 
    Document, 
    VectorStoreIndex, 
    SimpleDirectoryReader,
    RouterQueryEngine,
    storageContextFromDefaults,
    ContextChatEngine
} from "npm:llamaindex@0.1.12"
const keys = "sk-**" // read API key from .env

## 1. Persisting your data

### Set up a storage context

In [8]:
const storageContext = await storageContextFromDefaults({
  persistDir: "./storage",
});

No valid data found at path: storage/doc_store.json starting new store.
No valid data found at path: storage/index_store.json starting new store.
No valid data found at path: ./storage/vector_store.json starting new store.


### Load the data and create an index

In [ ]:
process.env.OPENAI_API_KEY = keys

In [11]:
const documents = await new SimpleDirectoryReader().loadData({
  directoryPath: "./data", // we have the React wikipedia page in here
});
let index = await VectorStoreIndex.fromDocuments(documents, {
  storageContext
});

### Let's ask it a question!

In [13]:
let engine = await index.asQueryEngine()
let response2 = await engine.query({query:"What is JSX?"})
console.log(response.toString())

[object Response]


### Get an index without parsing the documents 

In [14]:
let storageContext2 = await storageContextFromDefaults({
    persistDir: "./storage",
});

### Initialize the index

In [15]:
let index2 = await VectorStoreIndex.init({
    storageContext: storageContext2
});

### Let's query it!

In [16]:
let engine2 = await index2.asQueryEngine()
let response3 = await engine2.query({query:"What is JSX?"})
console.log(response2.toString())

JSX is a syntax extension for JavaScript that allows developers to write HTML-like code within their JavaScript files when working with React. It makes it easier to write and understand React components by combining HTML and JavaScript in a single file.


## 2. Chatting with your data

### Create a retriever and a new chat engine

In [17]:
const retriever = index2.asRetriever();
retriever.similarityTopK = 3;

let chatEngine = new ContextChatEngine({
    retriever,
});


3

### Let's try it!

In [18]:
let messageHistory = [
    {
        role: "user",
        content: "What is JSX?"
    },
    {
        role: "assistant",
        content: "JSX stands for JavaScript Syntax Extension. It is an extension to the JavaScript language syntax that provides a way to structure component rendering using syntax familiar to many developers. JSX is similar in appearance to HTML and is typically used to write React components, although components can also be written in pure JavaScript. It was created by Facebook and is similar to another extension syntax created for PHP called XHP."
    }
]

In [19]:
let newMessage = "What was that last thing you mentioned?"


In [22]:
const response4 = await chatEngine.chat({
  message: newMessage,
  chatHistory: messageHistory
});
console.log(response4.toString())

The last thing I mentioned was XHP. XHP is an extension syntax for PHP that was also created by Facebook. It allows developers to write PHP code using XML-like syntax, making it easier to work with complex HTML structures and maintain code readability. XHP is primarily used in the context of building web applications with PHP.


## 3. Streaming

### Setting stream to true!

In [23]:
const response5 = await chatEngine.chat({
  message: newMessage,
  chatHistory: messageHistory,
  stream: true,
});
console.log(response4)

Response {
  metadata: {},
  response: "The last thing I mentioned was XHP. XHP is an extension syntax for PHP that was also created by Facebook. It allows developers to write PHP code using XML-like syntax, making it easier to work with complex HTML structures and maintain code readability. XHP is primarily used in the context of building web applications with PHP.",
  sourceNodes: [
    TextNode {
      id_: "ee24574a-538e-4858-86cd-0191d212cf08",
      metadata: {},
      excludedEmbedMetadataKeys: [],
      excludedLlmMetadataKeys: [],
      relationships: {
        SOURCE: {
          nodeId: "./data/abramov.txt",
          metadata: {},
          hash: "HPM6aKQO4S9cxAZf/SEinz9ZkBg8PvCKV7D6KypcnQk="
        },
        PREVIOUS: {
          nodeId: "6af9af6d-94f6-4176-8796-77138c5e695c",
          metadata: {},
          hash: "SfOgFxgwrwu5FH4XRf3DltHDqkbdE7PTNZ/veWRElWw="
        },
        NEXT: {
          nodeId: "ae934730-f9b7-488c-9291-5b8f0c8889a6",
          metadata: {},
 

### Check the response!

In [25]:
for await (const data of response5) {
    console.log(data.response); // Print the data
}


The
 last
 thing
 I
 mentioned
 was
 the
 twenties
.
 In
 the
 context
 of
 the
 passage
,
 it
 refers
 to
 the
 next
 decade
,
 specifically
 the
 
202
0
s
.
 The
 author
 is
 expressing
 their
 anticipation
 and
 excitement
 for
 the
 future
 and
 the
 opportunities
 it
 holds
 for
 them
 to
 continue
 building
 and
 explaining
 things
.



## Create Llama App

### Let's see all three techniques in a real web app!

We will use the utility functions in **utils.ts** file. 

**Note:** Go to `File` and click on `Open` to access the **utils.ts** file and all the code used in this lesson.

In [26]:
import { runCreateLlamaApp } from './utils.ts'

**Note**: The below code might take a minute to complete. Once done, please click on the link to access the web app!

In [28]:
// runCreateLlamaApp()

### Demonstrate streaming to the UI

Please note that the link generated above is specific to your notebook and will display the web app set up with our data! 

Ask it some questions about React. You'll see it instantly answers with a stream.